In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent.parent
sys.path.append(str(BASE_DIR))

In [3]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case4
import numpy as np
import pandas as pd
from pathlib import Path
import subprocess
import os
import win32com.client as win32
import time
import random
import subprocess
import psutil
from itertools import product

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

def kill_by_pid(process_name):
   for proc in psutil.process_iter():
       if proc.name() == process_name:
          p = psutil.Process(proc.pid)
          p.terminate()

In [4]:
random_state = np.random.RandomState(2)
def lhs_sample(n_samples=1):
    bounds_values = bounds=[[40,100],[30,80],[3,10],[20,80],[0.1,5]]
    n_dimensions = len(bounds_values)
    result = np.empty((n_samples, n_dimensions))
    
    for i in range(n_dimensions):
        lower, upper = bounds_values[i]
        interval = np.linspace(lower, upper, n_samples + 1)
        points = (interval[:-1] + interval[1:]) / 2
        random_state.shuffle(points)
        result[:, i] = points
    return result
def sample(n_samples):
    lhs_data = lhs_sample(n_samples=n_samples)
    return lhs_data
init_points_array = sample(10)
init_points_array

array([[67.   , 37.5  ,  6.15 , 53.   ,  2.795],
       [49.   , 62.5  ,  9.65 , 77.   ,  1.815],
       [73.   , 32.5  ,  6.85 , 71.   ,  4.265],
       [43.   , 42.5  ,  3.35 , 23.   ,  3.285],
       [85.   , 47.5  ,  8.95 , 35.   ,  3.775],
       [55.   , 77.5  ,  4.05 , 29.   ,  0.345],
       [61.   , 67.5  ,  4.75 , 41.   ,  0.835],
       [79.   , 72.5  ,  7.55 , 65.   ,  4.755],
       [97.   , 52.5  ,  8.25 , 59.   ,  2.305],
       [91.   , 57.5  ,  5.45 , 47.   ,  1.325]])

In [5]:
file_location = BASE_DIR/'extractive_distillation'
DIR = BASE_DIR/'scripts_res'/'Algorithms'
DIR

WindowsPath('d:/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res/Algorithms')

In [ ]:
**

In [5]:
sol = 'ASBO'
case_name = 'Case4'
runs_number = 10

In [ ]:
obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
all_res_list = []

def obj_fun(x0,x1,x2,x3,x4):
    x = [x0,x1,x2,x3,x4,]
    obj,purities = Case4(x)
    return -obj,purities

nc_lb = [0.98,0.98]
nc_ub = [1,   1 ]
cons = NonlinearConstraint(obj_fun, nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.0001)

pbounds = {'x0':(40,100),'x1': (30, 80),'x2': (3,10), 'x3': (20,80),'x4': (0.1,5)}
for i in range(runs_number):
    print(i)
    kill_by_pid('AspenPlus.exe')
    time.sleep(1) 
    optimizer = BayesianOptimization(f=obj_fun, pbounds=pbounds,constraint=cons,verbose=2,random_state=i,relax_margin=1e-3,)
    acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)
    try:
        optimizer.maximize(init_points=100, n_iter=60,p_hyper=[[5, 2, 0.78, 0.95,1e-5]],acquisition_function=acquisition_function,allow_duplicate_points=True)
        print(optimizer.max)
        x_1_list.extend([optimizer.max['params']['x0']])
        x_2_list.extend([optimizer.max['params']['x1']])
        x_3_list.extend([optimizer.max['params']['x2']])
        x_4_list.extend([optimizer.max['params']['x3']])
        x_5_list.extend([optimizer.max['params']['x4']])  
        obj_list.extend([optimizer.max['target']])                                  
        con_1_list.extend([optimizer.max['constraint'][0]])
        con_2_list.extend([optimizer.max['constraint'][1]])
        all_res = [item['target'] for item in optimizer.res]
        all_fea = [item['allowed'] for item in optimizer.res]
        modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
        all_res_list.append(modified_res)      
    except Exception as e:
        print(e)
        obj_list.append(None)
        x_1_list.append(None)
        x_2_list.append(None)
        x_3_list.append(None)
        x_4_list.append(None)
        x_5_list.append(None)                        
        con_1_list.append(None)
        con_2_list.append(None)
        modified_res = [-np.inf for _ in range(100+60)]
        all_res_list.append(modified_res)   
    df2 = pd.DataFrame(all_res_list, columns=[list(range(100+60))])               
    df = pd.DataFrame({
    'obj': obj_list,
    'x_1': x_1_list,
    'x_2': x_2_list,
    'x_3': x_3_list,
    'x_4': x_4_list,
    'x_5': x_5_list,
    'con_1': con_1_list,
    'con_2': con_2_list,
    })
    df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)    

0
|   iter    |  target   |  allowed  |    x0     |    x1     |    x2     |    x3     |    x4     |
-------------------------------------------------------------------------------------------------
| 1         | -464.2    | False     | 55.9      | 39.25     | 5.975     | 52.7      | 4.486     |
| 2         | -1e+04    | False     | 91.9      | 44.75     | 9.755     | 70.7      | 2.869     |
| 3         | -1e+04    | False     | 41.5      | 62.25     | 7.095     | 40.1      | 1.938     |
| 4         | -1e+04    | False     | 73.3      | 76.25     | 4.015     | 52.1      | 2.378     |
| 5         | -520.0    | False     | 85.3      | 66.25     | 8.075     | 21.5      | 0.2225    |
| 6         | -1e+04    | False     | 96.1      | 73.75     | 3.315     | 49.7      | 4.534     |
| 7         | -243.9    | False     | 49.9      | 32.75     | 6.325     | 49.1      | 0.7615    |
| 8         | -1e+04    | False     | 84.1      | 37.75     | 7.515     | 69.5      | 0.8595    |
| 9         | -378

In [ ]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,x_3,x_4,x_5,con_1,con_2
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,-268.121006,70.159437,50.260748,5.000158,30.441772,0.701500,0.986367,0.986625
std,16.606317,9.505057,0.285019,0.397958,6.505441,0.334468,0.003002,0.006602
min,-299.598605,57.818925,49.795001,4.531670,23.621295,0.410474,0.980940,0.979232
25%,-274.578202,61.887109,50.064703,4.726314,25.018819,0.541333,0.984355,0.981029
50%,-260.375585,68.393259,50.301577,4.916475,29.231447,0.597860,0.986440,0.985402
75%,-258.275995,78.782584,50.445784,5.228854,34.611585,0.645641,0.988186,0.992152
max,-248.619251,82.556745,50.732763,5.643078,42.554715,1.560392,0.990731,0.997459


In [6]:
def obj_fun(x0,x1,x2,x3,x4):
    x = [x0,x1,x2,x3,x4,]
    obj,purities = Case4(x)
    return -obj,purities
nc_lb = [0.98,0.98]
nc_ub = [1,   1 ]
cons = NonlinearConstraint(obj_fun, nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.0001)
pbounds = {'x0':(40,100),'x1': (30, 80),'x2': (3,10), 'x3': (20,80),'x4': (0.1,5)}
kill_by_pid('AspenPlus.exe')
time.sleep(1) 
optimizer = BayesianOptimization(f=obj_fun, pbounds=pbounds,constraint=cons,verbose=2,random_state=0,relax_margin=1e-3,)
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)
optimizer.maximize(init_points_array = init_points_array, init_points=100, n_iter=60,p_hyper=[[5, 2, 0.78, 0.95,1e-5]],acquisition_function=acquisition_function,allow_duplicate_points=True)
print(optimizer.max)

|   iter    |  target   |  allowed  |    x0     |    x1     |    x2     |    x3     |    x4     |
-------------------------------------------------------------------------------------------------
| 1         | -375.3    | False     | 67.0      | 37.5      | 6.15      | 53.0      | 2.795     |
| 2         | -1e+04    | False     | 49.0      | 62.5      | 9.65      | 77.0      | 1.815     |
| 3         | -1e+04    | False     | 73.0      | 32.5      | 6.85      | 71.0      | 4.265     |
| 4         | -214.0    | False     | 43.0      | 42.5      | 3.35      | 23.0      | 3.285     |
| 5         | -535.3    | False     | 85.0      | 47.5      | 8.95      | 35.0      | 3.775     |
| 6         | -1e+04    | False     | 55.0      | 77.5      | 4.05      | 29.0      | 0.345     |
| 7         | -1e+04    | False     | 61.0      | 67.5      | 4.75      | 41.0      | 0.835     |
| 8         | -1e+04    | False     | 79.0      | 72.5      | 7.55      | 65.0      | 4.755     |
| 9         | -1e+04

In [ ]:
**

In [ ]:
sol = 'slsqp'
case_name = 'Case4'
runs_number = 10

In [ ]:
initial_x1 = []
initial_x2 = []
initial_x3 = []
initial_x4 = []
initial_x5 = []

x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
obj_list = []

initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []

def fun(x):
    global itera
    kill_by_pid('AspenPlus.exe')
    time.sleep(1)    
    obj,_cons = Case4(x)
    return obj,_cons[0],_cons[1]

def recyc(x):
    obj_fun = lambda x: fun(x)[0]
    nc_lb = [0.98,0.98]
    nc_ub = [1,   1]
    cons = NonlinearConstraint(lambda x: fun(x)[1:3], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    x0 =x

    lb = [40,  30,   3,   20,  0.1] 
    ub = [100, 80,  10,   80,  5]
    bd = Bounds(lb, ub, True)  

    result = minimize(obj_fun, x0, method='slsqp', bounds=bd, constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)

    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
    x_3_list.append(result.x[2])
    x_4_list.append(result.x[3])
    x_5_list.append(result.x[4])        

    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    initial_x3.append(x[0][2])  
    initial_x4.append(x[0][3])  
    initial_x5.append(x[0][4])      
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)

    df = pd.DataFrame({ 
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'initial_x3': initial_x3,
        'initial_x4': initial_x4,  
        'initial_x5': initial_x5,                 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })  
    df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

for combination in product(sample(10)):
    recyc(combination)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_21364\2255095782.py:38: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(obj_fun, x0, method='slsqp', bounds=bd, constraints=cons)
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


obj =  10000 , x =  [73.    37.5    8.25  77.     4.755]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21364\2255095782.py:38: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(obj_fun, x0, method='slsqp', bounds=bd, constraints=cons)
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


obj =  10000 , x =  [67.    62.5    7.55  41.     0.835]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21364\2255095782.py:38: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(obj_fun, x0, method='slsqp', bounds=bd, constraints=cons)
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


...

obj =  242.9095663 , x =  [85.    32.5    6.15  29.     1.815]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21364\2255095782.py:38: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(obj_fun, x0, method='slsqp', bounds=bd, constraints=cons)
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


obj =  10000 , x =  [91.    47.5    4.75  71.     3.285]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21364\2255095782.py:38: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(obj_fun, x0, method='slsqp', bounds=bd, constraints=cons)
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


obj =  10000 , x =  [97.    77.5    8.95  47.     3.775]


In [ ]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
false_count = (df['success'] == False).sum()
print(f"Count of rows where 'success' is False: {false_count}")
df_filtered = df[df['success'] != False]
df_filtered.describe()

NaN count in column 'obj': 0
Count of rows where 'success' is False: 10


,obj,x_1,x_2,x_3,x_4,x_5,initial_x1,initial_x2,initial_x3,initial_x4,initial_x5,nfev
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
**

In [ ]:
sol = 'SQP'
case_name = 'Case4'
runs_number = 10

In [ ]:
# Has to change aspen file location in Case 4
def obj_fun(x):
    kill_by_pid('AspenPlus.exe')
    time.sleep(1)     
    res = Case4(x[0])
    return res
obj_list = []
for combination in product(init_points_array):
    print(combination)
    res = obj_fun(combination)
    obj_list.append(res[0])
    print(res)
print(sum(obj_list),max(obj_list),min(obj_list))

(array([67.   , 37.5  ,  6.15 , 53.   ,  2.795]),)
(10000, [0, 0])
(array([49.   , 62.5  ,  9.65 , 77.   ,  1.815]),)
(230.83195031, [0.980895691, 0.980864222])
(array([73.   , 32.5  ,  6.85 , 71.   ,  4.265]),)
(10000, [0, 0])
(array([43.   , 42.5  ,  3.35 , 23.   ,  3.285]),)
(10000, [0, 0])
(array([85.   , 47.5  ,  8.95 , 35.   ,  3.775]),)
(10000, [0, 0])
(array([55.   , 77.5  ,  4.05 , 29.   ,  0.345]),)
(10000, [0, 0])
(array([61.   , 67.5  ,  4.75 , 41.   ,  0.835]),)
(10000, [0, 0])
(array([79.   , 72.5  ,  7.55 , 65.   ,  4.755]),)
(229.42046082999997, [0.980761464, 0.980757079])
(array([97.   , 52.5  ,  8.25 , 59.   ,  2.305]),)
(229.42809315999997, [0.980932304, 0.980833032])
(array([91.   , 57.5  ,  5.45 , 47.   ,  1.325]),)
(10000, [0, 0])
70689.68050429999 10000 229.42046082999997


In [7]:
sol = 'BOBYQA'
case_name = 'Case4'
runs_number = 10

In [8]:
# Has to change aspen file location in Case 4
def obj_fun(x):
    kill_by_pid('AspenPlus.exe')
    time.sleep(1)         
    res = Case4(x[0])
    return res
obj_list = []
for combination in product(init_points_array):
    print(combination)
    res = obj_fun(combination)
    obj_list.append(res[0])
    print(res)
print(sum(obj_list),max(obj_list),min(obj_list))
pd.DataFrame(obj_list).to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

(array([67.   , 37.5  ,  6.15 , 53.   ,  2.795]),)
(263.3800917, [0.981622228, 0.981000061])
(array([49.   , 62.5  ,  9.65 , 77.   ,  1.815]),)
(475.8700343, [0.9925769, 0.98100004])
(array([73.   , 32.5  ,  6.85 , 71.   ,  4.265]),)
(228.74522027, [0.980999987, 0.98017697])
(array([43.   , 42.5  ,  3.35 , 23.   ,  3.285]),)
(274.9444347, [0.98104466, 0.981000182])
(array([85.   , 47.5  ,  8.95 , 35.   ,  3.775]),)
(309.8520276, [0.980999962, 0.980824407])
(array([55.   , 77.5  ,  4.05 , 29.   ,  0.345]),)
(241.09371667, [0.980531424, 0.980999936])
(array([61.   , 67.5  ,  4.75 , 41.   ,  0.835]),)
(255.4986929, [0.98102808, 0.981000073])
(array([79.   , 72.5  ,  7.55 , 65.   ,  4.755]),)
(238.66639741, [0.981694881, 0.981000053])
(array([97.   , 52.5  ,  8.25 , 59.   ,  2.305]),)
(504.10940299999993, [0.992092052, 0.980999995])
(array([91.   , 57.5  ,  5.45 , 47.   ,  1.325]),)
(240.91267755, [0.981000022, 0.980889606])
3033.0726961 504.10940299999993 228.74522027


In [9]:
kill_by_pid('AspenPlus.exe')